# Buscador de libros

Vamos a aprovechar la fuerza de MUSE, el embedding multilingüe de Facebook para realizar búsquedas de libros a partir de una base de datos que contiene títulos de libros y resúmenes de los mismos.

## Carga de datos

In [2]:
import requests
import tarfile
import os

# URL del archivo comprimido
url = "https://www.cs.cmu.edu/~dbamman/data/booksummaries.tar.gz"

# Nombre del archivo comprimido
nombre_archivo_comprimido = "booksummaries.tar.gz"

Esta celda puede tardar un rato en ejecutar porque estamos descargando una gran cantidad de datos:

In [3]:
# Descargar el archivo
respuesta = requests.get(url)
print(respuesta)
# Verificar si la descarga fue exitosa
if respuesta.status_code == 200:
    # Guardar el contenido descargado en un archivo local
    with open(nombre_archivo_comprimido, "wb") as archivo:
        archivo.write(respuesta.content)


<Response [200]>


Observamos que hemos descargado el archivo a nuestro directorio de trabajo.
A continuación se crea la carpeta libros en la que vamos a guardar el contenido descomprimido del archivo:

In [4]:
!mkdir libros

__Nota.__ Si estás trabajando en local puedes descomprimir el archivo directamente. Si estás ejecutando en Colab descárgate el archivo para descomprimirlo.

In [5]:
# Nombre del archivo comprimido
nombre_archivo = 'booksummaries.tar.gz'

# Directorio de destino para la extracción
directorio_destino = './libros/'

# Abrir el archivo tar.gz en modo lectura
with tarfile.open(nombre_archivo, 'r') as archivo_tar:
    # Extraer todos los archivos en el directorio de destino
    archivo_tar.extractall(path=directorio_destino)

print("Descompresión exitosa.")

Descompresión exitosa.


Podemos observar como en la carpeta libros ya tenemos nuestro archivo .txt con todos los datos que vamos a utilizar. Con unas pocas líneas de código podemos observar las primeras líneas del archivo:

In [6]:
# Nombre del archivo
nombre_archivo = "./libros/booksummaries/booksummaries.txt"

# Número de líneas que deseas leer
num_lineas = 5

# Abrir el archivo en modo lectura
with open(nombre_archivo, "r") as archivo:
    # Leer las primeras num_lineas líneas
    for i in range(num_lineas):
        linea = archivo.readline()
        # Verificar si la línea no está vacía
        if linea:
            print(linea.strip())  # Imprimir la línea sin el carácter de nueva línea
        else:
            break  # Si llegamos al final del archivo, salimos del bucle


620	/m/0hhy	Animal Farm	George Orwell	1945-08-17	{"/m/016lj8": "Roman \u00e0 clef", "/m/06nbt": "Satire", "/m/0dwly": "Children's literature", "/m/014dfn": "Speculative fiction", "/m/02xlf": "Fiction"}	 Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a philosophy. The animals revolt and drive the drunken and irresponsible Mr Jones from the farm, renaming it "Animal Farm". They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal". Snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly. The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. Napoleon takes the pups from the farm dogs an

Cargamos los datos en una tabla:

In [7]:
import pandas as pd
# Leer el archivo en un DataFrame de Pandas
datos = pd.read_csv(nombre_archivo, sep='\t', header=None)

# Asignar nombres a las columnas
datos.columns = ['Indice', 'ID', 'Libro', 'Autor', 'Fecha de Publicación', 'Género', 'Resumen']

# Mostrar las primeras filas del DataFrame
print(datos.head())

   Indice       ID                                      Libro  \
0     620  /m/0hhy                                Animal Farm   
1     843  /m/0k36                         A Clockwork Orange   
2     986  /m/0ldx                                 The Plague   
3    1756  /m/0sww  An Enquiry Concerning Human Understanding   
4    2080  /m/0wkt                       A Fire Upon the Deep   

             Autor Fecha de Publicación  \
0    George Orwell           1945-08-17   
1  Anthony Burgess                 1962   
2     Albert Camus                 1947   
3       David Hume                  NaN   
4     Vernor Vinge                  NaN   

                                              Género  \
0  {"/m/016lj8": "Roman \u00e0 clef", "/m/06nbt":...   
1  {"/m/06n90": "Science Fiction", "/m/0l67h": "N...   
2  {"/m/02m4t": "Existentialism", "/m/02xlf": "Fi...   
3                                                NaN   
4  {"/m/03lrw": "Hard science fiction", "/m/06n90...   

             

Conservamos únicamente la información que nos interesa: el título del libro y el resumen.

In [8]:
libros_resumen = datos[["Libro", "Resumen"]]

In [10]:
libros_resumen.head(12)

,Libro,Resumen
0,Animal Farm,"Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,"Alex, a teenager living in near-future Englan..."
2,The Plague,The text of The Plague is divided into five p...
3,An Enquiry Concerning Human Understanding,The argument of the Enquiry proceeds by a ser...
4,A Fire Upon the Deep,The novel posits that space around the Milky ...
5,All Quiet on the Western Front,"The book tells the story of Paul Bäumer, a Ge..."
6,A Wizard of Earthsea,"Ged is a young boy on Gont, one of the larger..."
7,Anyone Can Whistle,The story is set in an imaginary American tow...
8,Blade Runner 3: Replicant Night,"Living on Mars, Deckard is acting as a consul..."
9,Blade Runner 2: The Edge of Human,Beginning several months after the events in ...


## Carga del embedding y construcción del buscador

Para usar MUSE necesitamos instalar tensorflow-text, un módulo de Google para Deep Learning en textos:

In [11]:
#@title
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 33.4 MB/s eta 0:00:00


A continuación importamos los módulos necesarios:

In [12]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import pandas as pd
import json
import os
import requests
import warnings
warnings.filterwarnings('ignore')

Cargamos el embedding desde la suite de Google:

In [13]:
embedding = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

Veámos cómo podemos transformar un texto en un vector de números con una sola línea de código:

In [14]:
ejemplo = embedding('Esto es un ejemplo')
print(ejemplo)

tf.Tensor(
[[-5.53136505e-02  7.51735270e-03 -1.33971982e-02  8.54406133e-02
   4.89022136e-02  2.73253750e-02 -6.46049855e-03 -9.66203213e-03
   3.59891690e-02  2.02767942e-02  3.21702883e-02 -5.52996621e-02
  -2.79749054e-02 -3.26700136e-02  2.09495556e-02 -1.66090783e-02
  -2.36730054e-02 -1.10793849e-02 -3.42288539e-02 -7.36196479e-03
  -6.04443206e-03  1.06974415e-01  1.51025550e-02  2.22048862e-03
   5.11200204e-02  3.16233411e-02  2.68504024e-02  3.38717643e-03
   6.07594810e-02 -1.13522187e-02 -1.14737444e-01  2.91699544e-02
  -5.75973764e-02 -5.86276762e-02  1.30695216e-02  2.69653387e-02
   7.55034611e-02 -2.40348149e-02  5.63987643e-02 -8.88167769e-02
   1.03335818e-02  3.82852517e-02  5.19630872e-02 -4.73263487e-03
   5.08798100e-02 -2.47666007e-03  3.10360231e-02  7.41313025e-02
   1.49562405e-02  3.36607778e-03 -2.59210691e-02  4.39192448e-03
  -2.23775990e-02  2.75029913e-02  5.00163659e-02  3.55228484e-02
   2.59855278e-02  5.96971717e-03 -4.88507971e-02  4.87358309e-02

El producto escalar es una operación matemática que puede medir la cercanía entre vectores. Veámos cómo MUSE es capaz de detectar cercanía entre distintas palabras en distintos idiomas.

__Nota.__ Dos palabras serán más próximas cuanto más se aproxime a 1 su producto escalar.

In [15]:
np.inner(embedding('Hola'), embedding('Buenas'))

array([[0.74412894]], dtype=float32)

In [16]:
np.inner(embedding('Hola'), embedding('Saluts'))

array([[0.9244789]], dtype=float32)

Una vez hecho esto lo que vamos a hacer es calcular el embedding de los resúmenes. De esta forma agilizaremos los cálculos posteriores.

__Nota.__ Por motivos de agilidad entrenaremos solo con los 200 primeros libros, si cambiáramos el parámetro del head podríamos calcularlo con más libros (siempre que dispongamos de los recursos para ello).

In [17]:
libros_resumen.shape

(16559, 2)

In [18]:
libros_resumen = libros_resumen.head(200)

En la siguiente celda vamos a calcular los embeddings de cada resumen. Dependiendo del ordenador esto puede llevar hasta 10 minutos. Cuando la celda haya terminado verás el embedding de las cinco primeras filas.

In [19]:
libros_resumen.head()

,Libro,Resumen
0,Animal Farm,"Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,"Alex, a teenager living in near-future Englan..."
2,The Plague,The text of The Plague is divided into five p...
3,An Enquiry Concerning Human Understanding,The argument of the Enquiry proceeds by a ser...
4,A Fire Upon the Deep,The novel posits that space around the Milky ...


In [20]:
libros_resumen['Embedding'] = libros_resumen.Resumen.apply(lambda x: embedding(x))
libros_resumen.head()

,Libro,Resumen,Embedding
0,Animal Farm,"Old Major, the old boar on the Manor Farm, ca...","((tf.Tensor(-0.030938277, shape=(), dtype=floa..."
1,A Clockwork Orange,"Alex, a teenager living in near-future Englan...","((tf.Tensor(-0.05130157, shape=(), dtype=float..."
2,The Plague,The text of The Plague is divided into five p...,"((tf.Tensor(-0.03425107, shape=(), dtype=float..."
3,An Enquiry Concerning Human Understanding,The argument of the Enquiry proceeds by a ser...,"((tf.Tensor(-0.016552985, shape=(), dtype=floa..."
4,A Fire Upon the Deep,The novel posits that space around the Milky ...,"((tf.Tensor(-0.08740418, shape=(), dtype=float..."


Ya estamos preparados para construir nuestra función buscador. Esta función recibirá tres parámetros:
- busqueda_usuario que será una consulta que realizaremos
- base_datos la tabla sobre la que vamos a buscar con los embeddings ya calculados
- resultados_deseados que codifica el número de resultados que deseamos.

In [21]:
def buscador_libros(busqueda_usuario, base_datos, resultados_deseados):
  embedding_busqueda = embedding(busqueda_usuario) #calculamos el embedding de la consulta
  base_datos['grado_similaridad'] = base_datos.Embedding.apply(lambda x: np.inner(embedding_busqueda, x)) #calculamos el producto escalar de la consulta con cada resumen
  base_datos = base_datos.sort_values(by=['grado_similaridad'], ascending=False) # ordenamos priorizando aquellos productos escalares más altos
  resultado = base_datos.iloc[0 : resultados_deseados][['Libro', 'grado_similaridad']] # nos quedamos con los resultados deseados
  return resultado

Y aquí vamos a ver la magia de nuestro buscador y nuestro embedding multilingüe; podemos realizar búsquedas en distintos idiomas:

In [22]:
buscador_libros('vampire', libros_resumen, 3)

,Libro,grado_similaridad
146,I Am Legend,[[0.22629733]]
26,Dracula,[[0.1971609]]
20,Crash,[[0.18085138]]


In [23]:
buscador_libros('animales', libros_resumen, 5)

,Libro,grado_similaridad
0,Animal Farm,[[0.08214639]]
155,The Star Beast,[[0.06634684]]
30,Darwin's Dangerous Idea,[[0.043676686]]
77,Stuart Little,[[0.042603724]]
17,Book of Jonah,[[0.04100634]]


In [24]:
buscador_libros('héroe', libros_resumen, 3)

,Libro,grado_similaridad
31,Death of a Hero,[[0.24052513]]
132,Captains Courageous,[[0.15950987]]
47,Johnny Got His Gun,[[0.15065494]]


Te invito a realizar tus propias búsquedas con una o más palabras.

## Conclusión

En esta práctica hemos construido una función tan útil como es un buscador multilingüe en apenas 15 minutos. Esa es la fuerza de los embeddings preentrenados, podemos beneficiarnos de los modelos creados por expertos con gran disponibilidad de datos y recursos para nuestros propios fines. ¡Espero que te haya resultado útil y te animo a copiarte el cuaderno y probar con tus propias modificaciones e incluso con otras bases de datos.